In [53]:
pip install beautifulsoup4
pip install lxml

In [17]:
from bs4 import BeautifulSoup
#from urllib.request import urlopen
import requests
import pandas as pd
import numpy as np

In [117]:
with open ('postcode_of_canada.html') as html_file:
    soup = BeautifulSoup(html_file,'lxml')

In [2]:
resposne = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(resposne,'lxml')
print(soup.prettify()) 

In [10]:
table = soup.find('table',class_ ='wikitable sortable')
print(table.text)



Postcode
Borough
Neighborhood


M1A
Not assigned
Not assigned


M2A
Not assigned
Not assigned


M3A
North York
Parkwoods


M4A
North York
Victoria Village


M5A
Downtown Toronto
Harbourfront


M6A
North York
Lawrence Heights


M6A
North York
Lawrence Manor


M7A
Downtown Toronto
Queen's Park


M8A
Not assigned
Not assigned


M9A
Queen's Park
Not assigned


M1B
Scarborough
Rouge


M1B
Scarborough
Malvern


M2B
Not assigned
Not assigned


M3B
North York
Don Mills North


M4B
East York
Woodbine Gardens


M4B
East York
Parkview Hill


M5B
Downtown Toronto
Ryerson


M5B
Downtown Toronto
Garden District


M6B
North York
Glencairn


M7B
Not assigned
Not assigned


M8B
Not assigned
Not assigned


M9B
Etobicoke
Cloverdale


M9B
Etobicoke
Islington


M9B
Etobicoke
Martin Grove


M9B
Etobicoke
Princess Gardens


M9B
Etobicoke
West Deane Park


M1C
Scarborough
Highland Creek


M1C
Scarborough
Rouge Hill


M1C
Scarborough
Port Union


M2C
Not assigned
Not assigned


M3C
North York
Flemingdon Park

In [11]:

table_list = []

for i, tr in enumerate(table.find_all('tr')):
    if i != 0:
        tds = tr.find_all('td')
#        print(tds)
#       print(type(tds))
        table_list.append({
            'Postcode': tds[0].contents[0],
            'Borough':tds[1].contents[0],
            'Neighborhood':tds[2].contents[0]
            })
print(table_list)

In [259]:
table = soup.find('table',class_ ='wikitable sortable')
#print(table)

Postcode = []
Borough = []
Neighborhood = []

for i, tr in enumerate(table.find_all('tr')):
    if i != 0:
        tds = tr.find_all('td')
#        print(tds)
#       print(type(tds))
        Postcode.append(tds[0].text)
        Borough.append(tds[1].text)
        Neighborhood.append(tds[2].text)

#print(Postcode)

data = {'PostalCode': Postcode, 'Borough': Borough, 'Neighborhood': Neighborhood}
data

{'PostalCode': ['M1A',
  'M2A',
  'M3A',
  'M4A',
  'M5A',
  'M6A',
  'M6A',
  'M7A',
  'M8A',
  'M9A',
  'M1B',
  'M1B',
  'M2B',
  'M3B',
  'M4B',
  'M4B',
  'M5B',
  'M5B',
  'M6B',
  'M7B',
  'M8B',
  'M9B',
  'M9B',
  'M9B',
  'M9B',
  'M9B',
  'M1C',
  'M1C',
  'M1C',
  'M2C',
  'M3C',
  'M3C',
  'M4C',
  'M5C',
  'M6C',
  'M7C',
  'M8C',
  'M9C',
  'M9C',
  'M9C',
  'M9C',
  'M1E',
  'M1E',
  'M1E',
  'M2E',
  'M3E',
  'M4E',
  'M5E',
  'M6E',
  'M7E',
  'M8E',
  'M9E',
  'M1G',
  'M2G',
  'M3G',
  'M4G',
  'M5G',
  'M6G',
  'M7G',
  'M8G',
  'M9G',
  'M1H',
  'M2H',
  'M3H',
  'M3H',
  'M3H',
  'M4H',
  'M5H',
  'M5H',
  'M5H',
  'M6H',
  'M6H',
  'M7H',
  'M8H',
  'M9H',
  'M1J',
  'M2J',
  'M2J',
  'M2J',
  'M3J',
  'M3J',
  'M4J',
  'M5J',
  'M5J',
  'M5J',
  'M6J',
  'M6J',
  'M7J',
  'M8J',
  'M9J',
  'M1K',
  'M1K',
  'M1K',
  'M2K',
  'M3K',
  'M3K',
  'M4K',
  'M4K',
  'M5K',
  'M5K',
  'M6K',
  'M6K',
  'M6K',
  'M7K',
  'M8K',
  'M9K',
  'M1L',
  'M1L',
  'M1L',
  'M2

The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

In [260]:
df = pd.DataFrame(data)
df = df.replace('\n','', regex=True) 
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Ignore cells with a borough that is Not assigned.

In [261]:
df = df[df['Borough'] != 'Not assigned']
df.shape

(210, 3)

More than one neighborhood can exist in one postal code area

In [262]:
df = df.groupby(['PostalCode','Borough']).agg(lambda x: ','.join(x)).reset_index()
df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.

In [264]:
df['Neighborhood'].replace('Not assigned', "Queen's Park",inplace= True)
df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


use the .shape method to print the number of rows of your dataframe.

In [265]:
df.shape[0]

103